In [1]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta, time
from IPython.display import display

from atj_algotrading.backtester import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [4]:
symbol = 'US500'
start_dt = datetime(2015, 1, 1)
end_dt = datetime.now()

# get historical data
mt5.initialize()
ohlc = get_ohlc_history(symbol, mt5.TIMEFRAME_D1, start_dt, end_dt)
ohlc['sma'] = ohlc['open'].rolling(200).mean()

display(ohlc)

fig = create_price_fig(ohlc, indicators=['sma'])
display(fig)

,time,open,high,low,close,sma
0,2015-01-02,2063.00,2073.00,2044.70,2056.50,NaN
1,2015-01-05,2053.10,2054.00,2016.00,2024.50,NaN
2,2015-01-06,2024.70,2030.20,1991.50,2001.70,NaN
3,2015-01-07,2001.10,2030.00,2000.90,2027.00,NaN
4,2015-01-08,2027.20,2064.80,2026.70,2059.80,NaN
...,...,...,...,...,...,...
2517,2024-10-01,5752.29,5767.34,5681.16,5706.09,5251.43145
2518,2024-10-02,5707.84,5721.50,5673.16,5718.22,5256.23650
2519,2024-10-03,5717.89,5721.26,5675.49,5694.07,5261.06580
2520,2024-10-04,5695.86,5755.16,5691.41,5745.86,5265.65510


In [20]:
def get_candle_type(x):
    if x['close'] > x['open']:
        return 1 # bullish candle
    elif x['close'] < x['open']:
        return -1 # bearish candle
    else:
        return 0
    
ohlc['candle_type'] = ohlc.apply(get_candle_type, axis=1)

# We're checking 3 bearish candles in a row
ohlc['prev_candle_sequences'] = ohlc['candle_type'].rolling(3).sum().shift(1)
ohlc['gain'] = ohlc['close'] - ohlc['open']
ohlc['perc_gain'] = ohlc['gain'] / ohlc['open']
ohlc['prev_low'] = ohlc['low'].shift(1)

ohlc

,time,open,high,low,close,sma,candle_type,candle_sequences,prev_candle_sequences,gain,perc_gain,prev_low
0,2015-01-02,2063.00,2073.00,2044.70,2056.50,NaN,-1,NaN,NaN,-6.50,-0.003151,NaN
1,2015-01-05,2053.10,2054.00,2016.00,2024.50,NaN,-1,NaN,NaN,-28.60,-0.013930,2044.70
2,2015-01-06,2024.70,2030.20,1991.50,2001.70,NaN,-1,-3.0,NaN,-23.00,-0.011360,2016.00
3,2015-01-07,2001.10,2030.00,2000.90,2027.00,NaN,1,-1.0,-3.0,25.90,0.012943,1991.50
4,2015-01-08,2027.20,2064.80,2026.70,2059.80,NaN,1,1.0,-1.0,32.60,0.016081,2000.90
...,...,...,...,...,...,...,...,...,...,...,...,...
2517,2024-10-01,5752.29,5767.34,5681.16,5706.09,5251.43145,-1,-1.0,1.0,-46.20,-0.008032,5701.90
2518,2024-10-02,5707.84,5721.50,5673.16,5718.22,5256.23650,1,1.0,-1.0,10.38,0.001819,5681.16
2519,2024-10-03,5717.89,5721.26,5675.49,5694.07,5261.06580,-1,-1.0,1.0,-23.82,-0.004166,5673.16
2520,2024-10-04,5695.86,5755.16,5691.41,5745.86,5265.65510,1,1.0,-1.0,50.00,0.008778,5675.49


In [21]:
ohlc2 = ohlc[(ohlc['prev_candle_sequences'] == -3) & (ohlc['sma'] < ohlc['open'])].copy()

ohlc2['perc_gain'].sum()

0.10530493365283432

In [9]:
# 1) Price must be above SMA 200
# 2) Signal - 3 bearish candles in a row

In [22]:
# create trade logic
def on_bar(data, trades, orders):
    volume = round(10000 / data['open'], 2) # 1 lots
    
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    
    # entry signal
    if data['prev_candle_sequences'] == -3 and data['sma'] < data['open'] and not num_open_trades:
        orders.open_trade(symbol, volume, 'buy', sl=data['prev_low'])
        
    # exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]
        orders.close_trade(trade)

        

In [23]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -1

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(ohlc)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,US500,buy,4.77,2015-11-09 00:00:00,2097.8,2015-11-09 00:00:00,2083.5,2083.5,0,None,-68.21,-4.77,-72.98,-72.98,9927.02
1,closed,US500,buy,4.81,2015-11-10 00:00:00,2077.6,2015-11-11 00:00:00,2081.6,2067.5,0,None,19.24,-4.81,14.43,-58.55,9941.45
2,closed,US500,buy,4.88,2016-05-06 00:00:00,2049.2,2016-05-06 00:00:00,2045.4,2045.4,0,None,-18.54,-4.88,-23.42,-81.97,9918.03
3,closed,US500,buy,4.9,2016-05-20 00:00:00,2041.8,2016-05-23 00:00:00,2055.9,2024.9,0,None,69.09,-4.9,64.19,-17.78,9982.22
4,closed,US500,buy,4.82,2016-06-14 00:00:00,2076.2,2016-06-14 00:00:00,2075.8,2075.8,0,None,-1.93,-4.82,-6.75,-24.53,9975.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,closed,US500,buy,1.85,2024-07-26 00:00:00,5406.42,2024-07-29 00:00:00,5457.98,5390.89,0,None,95.39,-1.85,93.54,566.31,10566.31
120,closed,US500,buy,1.92,2024-08-06 00:00:00,5220.82,2024-08-07 00:00:00,5213.59,5089.71,0,None,-13.88,-1.92,-15.8,550.51,10550.51
121,closed,US500,buy,1.93,2024-08-08 00:00:00,5180.77,2024-08-08 00:00:00,5170.15,5170.15,0,None,-20.50,-1.93,-22.43,528.08,10528.08
122,closed,US500,buy,1.82,2024-09-06 00:00:00,5502.56,2024-09-06 00:00:00,5479.22,5479.22,0,None,-42.48,-1.82,-44.3,483.78,10483.78


In [24]:
pnl_chart = bt.plot_pnl()
pnl_chart

In [19]:
backtest_fig = bt.visualize_backtest(indicators=['sma'])
backtest_fig